In [2]:
from pymongo import MongoClient
import pandas as pd
from folium.vector_layers import Circle, Polygon
from folium import Map, Marker, Icon, FeatureGroup, LayerControl, Choropleth
import sources.functions.search_place.properties.client as mongo

## Mongo queries to get the following data:
- #### The 1M dollar companies based on San Francisco, California
- #### The design companies based on San Francisco, California
- #### The places searched by the google nearby place API having the following consideration:
    - Starbucks,
    - Vegan restaurants,
    - Night clubs,
    - Airports, and
    - pet groomers.


Noted that the consideration above are specific requirements for this project.

Now, this data will be shown in a map in order to have a visual understanding to 

In [3]:
filt = {
    'ipo.valuation_amount':{'$gte':1_000_000},
    'offices.city':'San Francisco'
}
project = {
    '_id':0,
    'name':1,
    'offices.latitude':1,
    'offices.longitude':1
}
co_1M = mongo.db.companies_unwinded.find(filt, project)

filt = {
    'category_code':'design', 
    'offices.city':'San Francisco'
}
project = {
    '_id':0,
    'name':1, 
    'offices.latitude':1,
    'offices.longitude':1
}
design_co = mongo.db.companies_unwinded.find(filt, project)

searched_places_1M = mongo.db.searched_places.find({})

In [4]:
centre = [37.7749,-122.4194]

In [5]:
m = Map(location=centre, zoom_start=11)

## Places near 1M-company offices, considering the categories below:
- ##### starbucks,
- ##### vegan restaurants,
- ##### dog grooming,
- ##### night clubs, and
- ##### airports

In [6]:
for place in searched_places_1M:
    
#     if place['name'] == 'Starbucks':
#         ic = Icon(color="blue",
#                   prefix="glyphicon",
#                   icon="glyphicon-signa",
#                   icon_color="black")
#     elif 'restaurant' in place['types']:
#         ic = Icon(color="blue",
#                   prefix="glyphicon",
#                   icon="glyphicon-apple",
#                   icon_color="black")
#     elif 'pet' in place['types']:
#         ic = Icon(color="green",
#                   prefix="fa",
#                   icon="fa-cc-mastercard",
#                   icon_color="black")
#     elif 'night club' in place['types']:
#         ic = Icon(color="blue",
#                   prefix="glyphicon",
#                   icon="glyphicon-volume-up",
#                   icon_color="black")
#     elif 'airport' in place['types']:
#         ic = Icon(color="blue",
#                   prefix="glyphicon",
#                   icon="glyphicon-plane",
#                   icon_color="black")
         
    Marker(location=[ place['coordinates']['lat'], place['coordinates']['lng'] ],
           tooltip=place['name'], popup=', '.join(place['types'])
          ).add_to(m)

## 1M-company offices

In [7]:
for office in co_1M:
    ic = Icon(color="red",
                  prefix="glyphicon",
                  icon="glyphicon-asterisk",
                  icon_color="black")
    Marker(location=[ office['offices']['latitude'], office['offices']['longitude'] ],
           tooltip=office['name'], icon=ic
          ).add_to(m)
    Circle(
        location=[ office['offices']['latitude'], office['offices']['longitude'] ],
        radius=250,
        popup=office['name'],
        color='#3186cc',
        fill=True,
        fill_color='#3186cc'
    ).add_to(m)

## Design company

In [8]:
for office in design_co:
    ic = Icon(color="blue",
                  prefix="glyphicon",
                  icon="glyphicon-pencil",
                  icon_color="black")
    Marker(location=[ office['offices']['latitude'], office['offices']['longitude'] ],
           tooltip=office['name'], icon=ic
          ).add_to(m)
    Circle(
        location=[ office['offices']['latitude'], office['offices']['longitude'] ],
        radius=250,
        popup=office['name'],
        color='#crimson',
        fill=True,
        fill_color='#crimson'
    ).add_to(m)

In [9]:
m

In [11]:
m.save('outputs/map.html')